# Benchmarking Our Algorithm against others

In [1]:
import sys
sys.path.append('../')
from panav.environment.env import Room
from panav.hybrid import HybridGraph
from panav.util import flowtime,makespan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning



from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

In [2]:

bloating_r = 0.5
vmax = 1.0

#### 100 agent configuration ####
n_agents = list(range(10,101,10))
bloating_r = 0.5
vmax = 1.0



TIMEOUT = 100 # timeout

# Algorithms
algs = {  
        "PIBT": lambda hg: PIBT_plan(hg,vmax,bloating_r,TIMEOUT,simple_plan=False),
        "S2M2": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Simple_MILP_Planning",TIMEOUT=TIMEOUT)[0],
        "PBS_Tube": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Tube_Planning",TIMEOUT=TIMEOUT)[0],
        "TAHP": lambda hg: TAHP(hg,vmax=vmax,bloating_r=bloating_r,TIMEOUT=TIMEOUT),
        "Seq_Tube": lambda hg: sequential_planning(Tube_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        # "Seq_Temp_Obs": lambda hg: sequential_planning(Simple_MILP_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT)
        }

timeouted = {alg:False for alg in algs}

results = []

env_name = "Room"


In [3]:
# Experiments
cur_N = 0
for i in range(len(n_agents)):
    
    N = n_agents[i]
   

    env = Room(N_agent=N)
    print('Constructing environment')
    HG = HybridGraph(env,agent_radius=bloating_r,tunnels=env.tunnels)

    if N<cur_N:
        continue
  
    # Save checkpoint data
    with open("./data/{}_misc.pkl".format(env_name),'wb') as fp:
        pkl.dump({"TIMEOUT":TIMEOUT,'cur_N':N},fp)
        
    for name,alg in algs.items():

        if timeouted[name]:
            continue
        
        print("Alg:", name, "N agent:",N)

        result = {}
        
        t0 = time()
        plan = alg(HG)
        t = time()-t0
        
        result['plan'] = plan
        result['time'] = t

        result['N_agent'] = N
        result['alg_name'] = name

        result["TIMEOUT"]=TIMEOUT
        result["HG"]=HG

        results.append(result)

        
        if t>TIMEOUT:
            timeouted[name] = True
            print("Time out for algorithm", name, "N agent = ", N)

    # Save checkpoint data
    with open("./data/{}.pkl".format(env_name),'wb') as fp:
        pkl.dump(results,fp)


Constructing environment


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


Alg: PIBT N agent: 10
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-15
Alg: S2M2 N agent: 10


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


PBS Time Elapsed: 24.521138191223145
PBS Time Elapsed: 26.355839252471924
PBS Time Elapsed: 28.686777114868164
PBS Time Elapsed: 31.778565168380737
PBS Time Elapsed: 33.66096520423889
Alg: PBS_Tube N agent: 10
PBS Time Elapsed: 26.608155250549316
PBS Time Elapsed: 28.425938844680786
PBS Time Elapsed: 30.773406982421875
PBS Time Elapsed: 34.16299295425415
PBS Time Elapsed: 35.65519404411316
Alg: TAHP N agent: 10
Planning for 0/10
Planning for 1/10
Problem becomes infeasible.
Alg: Seq_Tube N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
Constructing environment
Alg: PIBT N agent: 20
Alg: S2M2 N agent: 20
PBS Time Elapsed: 39.202391147613525
PBS Time Elapsed: 41.363553047180176
PBS Time Elapsed: 43.68730711936951
PBS Time Elapsed: 45.15288805961609
PBS Time Elapsed: 48.092965841293335
PBS

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


Time out for algorithm Seq_Tube N agent =  40
Constructing environment
Alg: PIBT N agent: 50
Alg: TAHP N agent: 50
Planning for 0/50
Problem becomes infeasible.
Constructing environment
Alg: PIBT N agent: 60
Potential deadlock 57
Deadlock resolved at theta= 0.7853981633974483
Potential deadlock 57
Deadlock resolved at theta= 0.7853981633974483
Potential deadlock 57
Deadlock resolved at theta= 0.7853981633974483
Potential deadlock 57
Deadlock resolved at theta= 0.7853981633974483
Potential deadlock 57
Deadlock resolved at theta= 0.7853981633974483
Alg: TAHP N agent: 60
Planning for 0/60
Problem becomes infeasible.
Constructing environment
Alg: PIBT N agent: 70
Potential deadlock 20
Deadlock resolved at theta= 0.7853981633974483
Potential deadlock 20
Deadlock resolved at theta= 0.7853981633974483
Alg: TAHP N agent: 70
Planning for 0/70
Problem becomes infeasible.
Time out for algorithm TAHP N agent =  70
Constructing environment
Alg: PIBT N agent: 80
Potential deadlock 16
Deadlock resolv